In [2]:
%load_ext autoreload
%autoreload 2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import copy

import sys
sys.path.insert(0, "../models/")
sys.path.insert(0, "../lidar/")
from relu_nn import FFReLUNet
from lidar import BatchLidarDataset

In [6]:
# Make the graph
N = 5
G = nx.erdos_renyi_graph(N, 0.6)
nx.is_connected(G)

True

In [14]:
# Setup models
shape = [2, 32, 32, 16, 16, 1]
base_model = FFReLUNet(shape)
n = torch.nn.utils.parameters_to_vector(base_model.parameters()).shape[0]

models = {i : copy.deepcopy(base_model) for i in range(N)}
t1 = torch.nn.utils.parameters_to_vector(models[0].parameters())

torch.Size([1969])

In [15]:
def primal_update(data, model, base_loss, dual, thj, rho, lr, max_its):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    for k in range(max_its):
        opt.zero_grad()
        
        yh = model.forward(data["x"])
        
        th = torch.nn.utils.parameters_to_vector(model.parameters())
        reg = torch.sum(torch.square(torch.cdist(th.reshape(1, -1), thj)))
        loss = base_loss(yh, data["y"]) + torch.dot(th, dual) + rho * reg

        loss.backward()
        opt.step()

def joint_loss(base_loss, testx, testy, models, N):
    with torch.no_grad():
        loss = 0.0
        for i in range(N):
            pred = models[i].forward(testx)
            loss += base_loss(pred, testy)
        return loss

In [20]:
# Setup data
num_beams = 20
beam_samps = 20
scan_dist = 0.2
num_scans = 60
img_dir = "../data/32_b.png"
bs = 1000 # number of forward passes per gradient compute

training_dataloaders = {}
for i in range(N):
    ds = BatchLidarDataset(img_dir, num_beams, scan_dist, beam_samps, num_scans)
    training_dataloaders[i] = torch.utils.data.DataLoader(ds, batch_size=bs)

In [ ]:
max_its_cadmm = 400
max_its_adam = 15
rho = 0.5
lr = 0.005

